# Анна Запорощенко

В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.


Мы будем работать по заранее определённому пайплайну:

1.  Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше)  и 10 отзывов для проверки качества.   3 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

2. Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек  - просто словами, писать улучшающий код не надо (1 балл)

6. Логичность и чистота кода (1 балл) 

Да, тут 12 баллов - два можно потерять.


В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

In [1]:
import urllib.request
from fake_useragent import UserAgent
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\VADIK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#Функция очищает список текстов от тегов и ненужных знаков
def parse(info: list) -> list:
    reviews = []
    for i in info:
        n = re.sub('<[A-Za-z\/][^>]*>|\r', '', i[1])
        reviews.append(re.sub('\n', '. ', n))
    return reviews

In [3]:
#Функция принимает ссылку и ищет сначала все отзывы с самой низкой оценкой, затем - с самой высокой. 
#Возвращает по 44 отзыва каждого вида общим списком.
def get_reviews(url: str) -> list:
    user_agent = UserAgent().chrome
    req = urllib.request.Request(url, headers={'User-Agent':user_agent})
    with urllib.request.urlopen(req) as response:
        html_content = response.read().decode('utf-8')
    bad_reviews = parse(re.findall('<b>1</b></span> из 10(.+?)jrReviewComment"><div >(.+?)</div>', html_content, re.DOTALL))
    good_reviews = parse(re.findall('<b>10</b></span> из 10(.+?)jrReviewComment"><div >(.+?)</div>', html_content, re.DOTALL))
    return bad_reviews[0:44] + good_reviews[0:44]

In [4]:
#Функция принимает список отзывов. Каждый отзыв заменяет на список лемм, которые точно состоят 
#из букв и не являются стоп-словами. Все слова приводятся к нижнему регистру. Функция возвращает матрицу
#со списками лемм для каждого отзыва.
#Вряд ли сработает на Google Colab, потому что Mystem.lemmatize() подвисает на нем
def preprocessing(reviews: list) -> np.array:
    russian_stopwords = stopwords.words('russian')
    m = Mystem()
    for review in tqdm(enumerate(reviews)):
        reviews[review[0]] = []
        for word in m.lemmatize(review[1]):
            if (word.isalpha())and(word not in russian_stopwords):
                reviews[review[0]].append(word.lower())
    return np.array(reviews)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(preprocessing(get_reviews('https://www.megacritic.ru/film/prometej')),
                                                    np.concatenate((np.zeros(44), np.ones(44))),
                                                    test_size=10/88,
                                                    random_state=0)

88it [01:42,  1.17s/it]


In [6]:
#Списки лемм из отрицательных и положительных отзывов X_train помещаются в два отдельных списка, 
#из них потом создаются частотные словари.Также создается частотный словарь по всем отзывам X_train.
bad_reviews = []
good_reviews = []
for y in enumerate(y_train):
    if y[1] == 0:
        bad_reviews.extend(X_train[y[0]])
    else:
        good_reviews.extend(X_train[y[0]])
bad_reviews = Counter(bad_reviews)
good_reviews = Counter(good_reviews)
reviews = Counter(sum(X_train.tolist(), []))

In [7]:
#Функция принимает частотные словари отрицательных и положительных отзывов, а также общий частотный словарь.
#Она создает словарь со словами, которые присутствуют и в положительных, и в отрицательных отзывах.
def check (bad_reviews: Counter, good_reviews: Counter, reviews: Counter) -> Counter:
    words = Counter()
    for word, value in reviews.items():
        if (word in bad_reviews.keys())and(word in good_reviews.keys()):
            words[word] = value
    return words

In [8]:
#Функция позволяет регулировать, насколько часто встречающиеся слова отправлять в функцию check.
def cutter (reviews: Counter, start: int, stop: int) -> Counter:
    new_reviews = Counter()
    for word, value in reviews.items():
        if (value >= start)and(value <= stop):
            new_reviews[word] = value
    return new_reviews

Можно проверить, есть ли какая-то частотность, при которой конкретная лемма встречается только в одном виде отзывов. Единица не дает нам какой-либо информации, с частотностью 19 на самом деле есть только 1 слово.

In [9]:
numbers = list(Counter(reviews.values()).keys())
numbers.sort()
for number in numbers:
    print('частотность слова ', number, ' количество повторов ', 
          len(check(bad_reviews, good_reviews, cutter(reviews, number, number))))

частотность слова  1  количество повторов  0
частотность слова  2  количество повторов  137
частотность слова  3  количество повторов  75
частотность слова  4  количество повторов  56
частотность слова  5  количество повторов  30
частотность слова  6  количество повторов  25
частотность слова  7  количество повторов  16
частотность слова  8  количество повторов  10
частотность слова  9  количество повторов  6
частотность слова  10  количество повторов  5
частотность слова  11  количество повторов  11
частотность слова  12  количество повторов  3
частотность слова  13  количество повторов  3
частотность слова  14  количество повторов  3
частотность слова  15  количество повторов  2
частотность слова  17  количество повторов  5
частотность слова  18  количество повторов  1
частотность слова  19  количество повторов  0
частотность слова  20  количество повторов  2
частотность слова  22  количество повторов  2
частотность слова  23  количество повторов  2
частотность слова  24  количество 

Зато можно увидеть, что слова перестают создавать больше количество "шума", когда встречаются более 6 раз. Ровно разделить все слова на "положительные" и "отрицательные" не получилось.

In [10]:
neutral = check(bad_reviews, good_reviews, reviews)

In [11]:
def get_words(neutral: Counter, polar: Counter) -> Counter:
    new = Counter()
    for word, value in polar.items():
        if word not in neutral:
            new[word] = value
    return new

In [12]:
bad_reviews = get_words(neutral, bad_reviews)
good_reviews = get_words(neutral, good_reviews)

Чтобы преобразовать тексты в вектора, будем исользовать метод bag of words.

In [13]:
#Функция склеивает леммы внутри каждого отзыва через пробел, чтобы тексты можно было передать в CountVectoriser()
def joined(massive: np.array) -> np.array:
    for review in enumerate(massive):
        massive[review[0]] = (' ').join(review[1])
    return massive

In [14]:
X_train = joined(X_train)
X_test = joined(X_test)

In [15]:
vectorizer = CountVectorizer()

Для общего словаря использую только X_train.

In [16]:
vectorizer.fit_transform(X_train)

<78x1985 sparse matrix of type '<class 'numpy.int64'>'
	with 3726 stored elements in Compressed Sparse Row format>

In [17]:
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [18]:
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [19]:
accuracy_score(lr_model.predict(X_test), y_test)

0.6

# Предложения
1) Можно убрать "шум" (нечастотные слова из общего словаря), и тогда, возможно, результат будет лучше.
2) Использовать не просто частотность, а tf idf было бы выгоднее: слова, которые одинаково часто встречаются и в положительных, и в отрицательных отзывах, но которые не были исключены из словаря как стоп-слова (та самая нейтральная лексика), имели бы маленький вес в векторах.